In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler ,LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
data=pd.read_csv('Churn_Modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

onehotEncoder=OneHotEncoder(handle_unknown='ignore')
geo_encoder=onehotEncoder.fit_transform(data[['Geography']]).toarray()
geo_encoder_df=pd.DataFrame(geo_encoder,columns=onehotEncoder.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1), geo_encoder_df],axis=1)

X=data.drop('Exited',axis=1)
y=data['Exited']

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#Scaling the feature

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [7]:
with open('hyper_label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)
with open('hyper_geo_hot_encoder.pkl','wb')as file:
    pickle.dump(onehotEncoder,file)
with open('hyper_scaler.pkl','wb')as file:
    pickle.dump(scaler,file)

In [19]:
## Define a fxn to create the model and try diff parameters

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [20]:
## Create a keras classifier

model=KerasClassifier(layers=1,neurons=3,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [21]:
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [ ]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3)
grid_result=grid.fit(X_train,y_train)

print("Best %f using %s" % (grid_result.best_score_,grid_result.best_params_))

C:\Users\MSI\AppData\Roaming\Python\Python312\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\MSI\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\MSI\AppData\Roaming\Python\Python312\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\MSI\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_sha